In [17]:
import pandas as pd
import librosa
import os
import numpy as np

In [69]:
def load_data(path_file : str):
    """
    Carga los audios con librosa y los convierte en un dataframe
    """
    data_song = os.path.join(path_file, "Song")
    data_speech = os.path.join(path_file, "Speech")

    labels = []
    data = []
    for actor in os.listdir(data_song):
        actor_path = os.path.join(data_song, actor)
        for file in os.listdir(actor_path):
            file_path = os.path.join(actor_path, file)
            # Load audio file with Librosa
            y, sr = librosa.load(file_path)
            # Apply MFCC transformation
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=1024)
            # Append to data
            data.append(mfcc)
            labels.append(actor_path[-2:])

    for actor in os.listdir(data_speech):
        actor_path = os.path.join(data_speech, actor)
        for file in os.listdir(actor_path):
            file_path = os.path.join(actor_path, file)
            # Load audio file with Librosa
            y, sr = librosa.load(file_path)
            # Apply MFCC transformation
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            # Append to data
            data.append(mfcc)
            labels.append(actor_path[-2:])

    return data, labels

data, labels = load_data("sound_dataset")

max_len = 1024
# Convert data and labels to numpy arrays
# max_len = 0 
# for d in data:
#     if d.shape[1] > max_len:
#         max_len = d.shape[1]
        
# Expand each MFCC to have the same shape by filling with zeros
for i in range(len(data)):
    data[i] = np.pad(data[i], ((0, 0), (0, max_len - data[i].shape[1])), mode='constant')

data_np = np.array(data)

# Create dataframe and add label to last column
df = pd.DataFrame(data_np.reshape(data_np.shape[0], data_np.shape[1] * data_np.shape[2]))

for i in range(len(labels)):
    df.loc[i, 'label'] = labels[i]

In [70]:
# Split dataframe into train and test sets
train = df.sample(frac=0.8, random_state=0)
test = df.drop(train.index)

# Save train and test sets to CSV
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)



In [68]:
data_np.shape

(2429, 13, 275)